# AI Expense Reporting Analysis

This notebook analyzes AI subscription expense reporting by comparing:
1. CS Monthly AI Subscriptions CSV - people who reported expenses
2. People and AI Info CSV - all team members

The goal is to calculate what percentage of team members reported AI expenses and identify name mismatches.

In [ ]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re

# Display settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## 1. Load Data

In [ ]:
# Load CS Monthly AI Subscriptions data
subscriptions_df = pd.read_csv('../data/CS Monthly AI Subscriptions.csv', skiprows=9)

# Load People and AI Info data (most recent version)
people_df = pd.read_csv('../data/CSP AI Culture and Learning_ Tracking - People and AI Info_2025-10-17.csv')

print(f"Subscriptions data shape: {subscriptions_df.shape}")
print(f"People data shape: {people_df.shape}")

## 2. Extract Names from Subscriptions Data

In [ ]:
# Look at the structure of subscriptions data
print("Subscriptions DataFrame columns:")
print(subscriptions_df.columns.tolist())
print("\nFirst few rows:")
subscriptions_df.head(10)

In [ ]:
# Extract names from the subscriptions data
# The second column contains names with employee IDs in format: "Name (ID)"
def extract_name_and_id(name_with_id):
    """Extract name and employee ID from format 'Name (ID)'"""
    if pd.isna(name_with_id) or name_with_id == '(blank)':
        return None, None
    
    # Match pattern: text followed by (number)
    match = re.match(r'^(.+?)\s*\(([^)]+)\)\s*$', str(name_with_id))
    if match:
        name = match.group(1).strip()
        emp_id = match.group(2).strip()
        return name, emp_id
    return str(name_with_id).strip(), None

# Get the column with names (second column, index 1)
name_column = subscriptions_df.columns[1]

# Extract names and IDs
subscriptions_df['extracted_name'], subscriptions_df['employee_id'] = zip(
    *subscriptions_df[name_column].apply(extract_name_and_id)
)

# Filter out rows without valid names
reported_expenses = subscriptions_df[subscriptions_df['extracted_name'].notna()].copy()

# Remove summary rows (like totals)
reported_expenses = reported_expenses[~reported_expenses['extracted_name'].str.contains('AI Subscription|Users|blank', na=False)]

print(f"\nNumber of people who reported expenses: {len(reported_expenses)}")
print("\nNames extracted from subscriptions data:")
print(reported_expenses[['extracted_name', 'employee_id']].head(10))

## 3. Extract Names from People and AI Info Data

In [ ]:
# Look at the People data structure
print("People DataFrame columns:")
print(people_df.columns.tolist())
print("\nFirst few rows:")
people_df.head(10)

In [ ]:
# Clean up people names - remove extra whitespace and filter valid entries
people_df['Name'] = people_df['Name'].str.strip()
all_team_members = people_df[people_df['Name'].notna() & (people_df['Name'] != '')].copy()

# Remove any invalid entries
all_team_members = all_team_members[~all_team_members['Name'].str.contains('@', na=False)]  # Remove email entries

print(f"\nTotal team members: {len(all_team_members)}")
print("\nSample team member names:")
print(all_team_members['Name'].head(10).tolist())

## 4. Normalize Names for Matching

In [ ]:
def normalize_name(name):
    """Normalize name by removing special characters, extra spaces, and converting to lowercase"""
    if pd.isna(name):
        return ''
    
    # Convert to string and lowercase
    name = str(name).lower()
    
    # Remove Chinese characters in parentheses
    name = re.sub(r'[（(][^)）]*[）)]', '', name)
    
    # Remove special characters except spaces and hyphens
    name = re.sub(r'[^a-z\s-]', '', name)
    
    # Normalize spaces
    name = ' '.join(name.split())
    
    return name.strip()

# Normalize names in both datasets
reported_expenses['normalized_name'] = reported_expenses['extracted_name'].apply(normalize_name)
all_team_members['normalized_name'] = all_team_members['Name'].apply(normalize_name)

print("\nSample normalized names from subscriptions:")
print(reported_expenses[['extracted_name', 'normalized_name']].head(10))

print("\nSample normalized names from team list:")
print(all_team_members[['Name', 'normalized_name']].head(10))

## 5. Match Names Using Fuzzy Matching

In [ ]:
def find_best_match(name, choices, threshold=85):
    """Find the best fuzzy match for a name"""
    if not name or not choices:
        return None, 0
    
    result = process.extractOne(name, choices, scorer=fuzz.token_sort_ratio)
    if result and result[1] >= threshold:
        return result[0], result[1]
    return None, result[1] if result else 0

# Create a list of all team member normalized names for matching
team_names_list = all_team_members['normalized_name'].tolist()
team_names_original = all_team_members['Name'].tolist()

# Find matches for each person who reported expenses
matches = []
for idx, row in reported_expenses.iterrows():
    reported_name = row['normalized_name']
    best_match, score = find_best_match(reported_name, team_names_list)
    
    # Get the original name from the team list
    original_match = None
    if best_match:
        match_idx = team_names_list.index(best_match)
        original_match = team_names_original[match_idx]
    
    matches.append({
        'subscription_name': row['extracted_name'],
        'normalized_subscription': reported_name,
        'matched_team_name': original_match,
        'normalized_matched': best_match,
        'match_score': score,
        'is_matched': score >= 85
    })

matches_df = pd.DataFrame(matches)

print(f"\nTotal expense reports: {len(matches_df)}")
print(f"Successfully matched: {matches_df['is_matched'].sum()}")
print(f"Unmatched: {(~matches_df['is_matched']).sum()}")

## 6. Review Matches and Mismatches

In [ ]:
# Show all matches with scores
print("\n" + "="*80)
print("ALL MATCHES (sorted by match score)")
print("="*80)
print(matches_df.sort_values('match_score', ascending=False)[[
    'subscription_name', 'matched_team_name', 'match_score', 'is_matched'
]])

In [ ]:
# Show unmatched or low-confidence matches
print("\n" + "="*80)
print("UNMATCHED OR LOW CONFIDENCE MATCHES (score < 85)")
print("="*80)
unmatched = matches_df[~matches_df['is_matched']]
if len(unmatched) > 0:
    print(unmatched[['subscription_name', 'matched_team_name', 'match_score']])
else:
    print("All names matched successfully!")

## 7. Calculate Reporting Statistics

In [ ]:
# Get unique matched team members who reported
matched_reporters = matches_df[matches_df['is_matched']]['matched_team_name'].unique()
total_team_members = len(all_team_members)
num_reporters = len(matched_reporters)
reporting_percentage = (num_reporters / total_team_members) * 100

print("\n" + "="*80)
print("REPORTING STATISTICS")
print("="*80)
print(f"Total team members: {total_team_members}")
print(f"Team members who reported expenses: {num_reporters}")
print(f"Reporting percentage: {reporting_percentage:.1f}%")
print(f"Team members who did NOT report: {total_team_members - num_reporters}")

## 8. Identify Team Members Who Did NOT Report

In [ ]:
# Find team members who did not report
all_team_names = set(all_team_members['Name'].tolist())
reporters_set = set(matched_reporters)
non_reporters = sorted(all_team_names - reporters_set)

print("\n" + "="*80)
print(f"TEAM MEMBERS WHO DID NOT REPORT EXPENSES ({len(non_reporters)})")
print("="*80)
for i, name in enumerate(non_reporters, 1):
    print(f"{i}. {name}")

## 9. Cross-Check with 'Expensed' Column

In [ ]:
# Check if the People and AI Info sheet has an 'Expensed' column
if 'Expensed' in people_df.columns:
    print("\n" + "="*80)
    print("CROSS-CHECK WITH 'EXPENSED' COLUMN IN PEOPLE DATA")
    print("="*80)
    
    # People marked as expensed in the People sheet
    marked_expensed = all_team_members[all_team_members['Expensed'].notna() & 
                                       (all_team_members['Expensed'] != '')]['Name'].tolist()
    
    print(f"\nPeople marked as 'Expensed' in People sheet: {len(marked_expensed)}")
    print(f"People who actually reported (from subscriptions): {num_reporters}")
    
    # Find discrepancies
    marked_but_not_reported = set(marked_expensed) - reporters_set
    reported_but_not_marked = reporters_set - set(marked_expensed)
    
    if marked_but_not_reported:
        print(f"\nMarked as 'Expensed' but NOT in subscription report ({len(marked_but_not_reported)}):")
        for name in sorted(marked_but_not_reported):
            print(f"  - {name}")
    
    if reported_but_not_marked:
        print(f"\nReported expenses but NOT marked in 'Expensed' column ({len(reported_but_not_marked)}):")
        for name in sorted(reported_but_not_marked):
            print(f"  - {name}")

## 10. Export Results for Manual Review

In [ ]:
# Export matches for manual review
output_path = '../outputs/expense_reporting_analysis.xlsx'

with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    # All matches
    matches_df.to_excel(writer, sheet_name='All Matches', index=False)
    
    # Unmatched names
    unmatched.to_excel(writer, sheet_name='Unmatched Names', index=False)
    
    # Non-reporters
    non_reporters_df = pd.DataFrame({'Name': non_reporters})
    non_reporters_df.to_excel(writer, sheet_name='Did Not Report', index=False)
    
    # Summary statistics
    summary_df = pd.DataFrame({
        'Metric': [
            'Total Team Members',
            'Reported Expenses',
            'Did Not Report',
            'Reporting Percentage',
            'Matched Names',
            'Unmatched Names'
        ],
        'Value': [
            total_team_members,
            num_reporters,
            total_team_members - num_reporters,
            f"{reporting_percentage:.1f}%",
            matches_df['is_matched'].sum(),
            (~matches_df['is_matched']).sum()
        ]
    })
    summary_df.to_excel(writer, sheet_name='Summary', index=False)

print(f"\nResults exported to: {output_path}")